In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
#app = dash.Dash(__name__)

# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 
                                           options=[
                                           {'label': 'All Site', 'value': 'Launch Site'},
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},   
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                           ],
                                          placeholder='Select a Lanuch Site',
                                          value = 'Launch Site',
                                          searchable=True,
                                          style={'width':'100%', 'padding':'3px','font-size':30, 'textAlign': 'left'}),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=500,
                                    marks={0: '0', 2500: '2500',
                                           5000: '5000', 7500: '7500',
                                           10000: '10000'},
                                    value=[2500, 8500],
                                    #dots = True, 
                                    #allowCross = False,
                                    #disabled = True,
                                    #pushable = 500,
                                    #updatemode = 'mouseup'
                                    #included = True,
                                    #vertical = False,
                                    #verticalHeight  = 900            
                                  ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'Launch Site':
        fig = px.pie(filtered_df, values = 'class', names = 'Launch Site', title = 'testing the pie chart')
        return fig
    else:
        sites = ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']
        if entered_site in sites:
            fig1 = px.pie(filtered_df[filtered_df['Launch Site']==entered_site], names = 'class', title = 'testing the pie chart')
        return fig1

    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])


def get_scatter_plot(entered_site, payload_mass):
    new_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_mass[0]) & (spacex_df['Payload Mass (kg)']<=payload_mass[1])]
    if entered_site == 'Launch Site':
        fig = fig=px.scatter(new_df, x='Payload Mass (kg)', y='class', 
              title='Correlation between Payload and Success for all Sites', 
              color ='Booster Version Category')
        return fig
    else:
        sites = ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']
        if entered_site in sites:
            fig1 = px.scatter(new_df[new_df['Launch Site']==entered_site], 
                              x='Payload Mass (kg)', y='class', 
                              title='Correlation between Payload and Success for all Sites', 
                              color ='Booster Version Category')
        return fig1
    
    
# Run the app
#if __name__ == '__main__':
    #app.run_server()
    
# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost", port=7645, debug=True)